In [5]:
import tensorflow as tf
import numpy as np
from scipy import misc
import matplotlib.pyplot as plt

from model import PSPNet101, PSPNet50
from tools import *


ModuleNotFoundError: No module named 'model'

# Set up paths for Images and Weights

In [ ]:
tf.reset_default_graph()
# TODO: Change these values to where your model files are
ADE20k_param = {'crop_size': [473, 473],
                'num_classes': 150, 
                'model': PSPNet50,
                'weights_path': './model/pspnet50-ade20k/model.ckpt-0'}
cityscapes_param = {'crop_size': [720, 720],
                    'num_classes': 19,
                    'model': PSPNet101,
                    'weights_path': './model/pspnet101-cityscapes/model.ckpt-0'}

IMAGE_MEAN = np.array((103.939, 116.779, 123.68), dtype=np.float32)
image_path = './test input/12.jpg'

# TODO: If you want to inference on indoor data, change this value to `ADE20k_param`
param = ADE20k_param

# Preprocess Image & Create Network

In [ ]:
# pre-proecess image
img_np, filename = load_img(image_path)
img_shape = tf.shape(img_np)
h, w = (tf.maximum(param['crop_size'][0], img_shape[0]), tf.maximum(param['crop_size'][1], img_shape[1]))
img = tf.to_float(preprocess(img_np, h, w))

print(img_shape)

In [ ]:
img_np, filename = load_img(image_path)
img_np.shape

In [ ]:
# Create network.
PSPNet = param['model']
net = PSPNet({'data': img}, is_training=False, num_classes=param['num_classes'])

# Predict segmentation image

In [ ]:
raw_output = net.layers['conv6']

# Predictions.
raw_output_up = tf.image.resize_bilinear(raw_output, size=[h, w], align_corners=True)
raw_output_up = tf.image.crop_to_bounding_box(raw_output_up, 0, 0, img_shape[0], img_shape[1])
raw_output_up = tf.argmax(raw_output_up, dimension=3)
pred = decode_labels(raw_output_up, img_shape, param['num_classes'])

# Init tf Session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
init = tf.global_variables_initializer()

sess.run(init)

ckpt_path = param['weights_path']
loader = tf.train.Saver(var_list=tf.global_variables())
loader.restore(sess, ckpt_path)
print("Restored model parameters from {}".format(ckpt_path))
    
# Run and get result image
preds = sess.run(pred)

In [ ]:
import matplotlib.patches as mpatches
plt.figure(1, [15, 30])
plt.subplot(121)
a_np=img_np.eval(session=sess)
plt.imshow(a_np)
plt.axis('off')
plt.subplot(122)
im = plt.imshow(preds[0]/255)
plt.axis('off')


plt.show()

In [ ]:

a_np=pred.eval(session=sess)
a_np.shape

In [ ]:
a=np.unique(a_np.reshape(-1,3),axis = 0)
len(a)
a[1][1]

# 建立颜色标签字典

In [ ]:
import csv
new_dict = {}
with open('D:/ruanjian/streetview/Image Segmentation/PSPNet-tensorflow-master/test/color label 150(2).csv')as csv_file:
    data = csv.reader(csv_file)
    new_dict = {row[0]: row[1:] for row in data}

new_dict


In [ ]:
new_dict['120120120'][0]

# 分割结果to_csv

In [ ]:
al = image_path.split('_')
data = [[al[2],al[3]]]
adf = pd.DataFrame(columns=['Lat','Lon'])
adf2= pd.DataFrame(data,columns=['Lat','Lon'])
adf=adf.append(adf2,ignore_index=True)
colours, counts = np.unique(a_np.reshape(-1,3), axis=0, return_counts=1)
i=0
for index, colour in enumerate(colours):
        count = counts[index]
        rgb = str(int(colour[0]*1000000+colour[1]*1000+colour[2]))
        adf[new_dict[rgb][0]]=pd.Series([count])
        #proportion = (100 * count) / (h * w)
        #print("Colour:",colour,"count:",count,'rgb:',rgb)
adf.head()

In [ ]:
adf.to_csv('D:/ruanjian/streetview/Image Segmentation/PSPNet-tensorflow-master/result_csv/last1.csv')

In [ ]:
import pandas as pd
df=pd.read_csv('D:/ruanjian/streetview/Image Segmentation/PSPNet-tensorflow-master/test/color labels.csv') 
df.head(10)

In [ ]:
df=pd.read_csv('D:/ruanjian/streetview/Image Segmentation/PSPNet-tensorflow-master/result_csv/Merge90-0.csv') 
df.head()

In [ ]:
df1=pd.read_csv('D:/ruanjian/streetview/Image Segmentation/PSPNet-tensorflow-master/result_csv/Merge270-0.csv') 
df1.head()

# DataFrame两表相加

In [ ]:
df2=df.add(df1,fill_value=0)
df2.to_csv('D:/ruanjian/streetview/Image Segmentation/PSPNet-tensorflow-master/result_csv/Merge270+90.csv')

# 两表融合（并集）

In [ ]:
df=pd.read_csv('D:/ruanjian/streetview/Image Segmentation/PSPNet-tensorflow-master/result_csv/Merge90-NAN.csv') 
df3 = pd.concat([df, adf], axis=0,sort = True)
df3.head()

In [ ]:
al = image_path.split('_')
al

In [ ]:
error=[]
error.append(al[2]+','+al[3])
error

In [ ]:
data = [[al[2],al[3]]]
adf = pd.DataFrame(columns=['Lat','Lon'])
adf2= pd.DataFrame(data,columns=['Lat','Lon'])
adf=adf.append(adf2)
#adf['102200']=pd.Series([27182])

adf.head()

In [ ]:
import imageio
imageio.imwrite('./output/cn_31.2031940000_121.4824660000_90.jpg', preds[0])
#misc.imsave('./output/cn_31.2031940000_121.4824660000.jpg', preds[0])

In [ ]:
np.unique(a_np[:,:,:,:],axis=1).shape

In [ ]:
values = np.unique(a_np.reshape(-1,3), axis=0)

colors = [ im.cmap(im.norm(value)) for value in values]
# create a patch (proxy artist) for every color 
patches = [ mpatches.Patch(color=colors[i], label="Level {l}".format(l=values[i]) ) for i in range(len(values)) ]
# put those patched as legend-handles into the legend
plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0. )